## Debugging: Launch Notebook in Browser

**For better visualization and debugging**, launch this notebook in a browser with Playwright automation hooks!


In [ ]:
# Launch this notebook in browser for better visualization & debugging
from playwright.sync_api import sync_playwright
import subprocess
import time
import sys
from pathlib import Path

notebook_file = Path("unifi_schema_database.ipynb")
notebook_name = notebook_file.name

# Check for Jupyter server
jupyter_url = None
try:
    import requests
    for port in [8888, 8889, 8890]:
        try:
            response = requests.get(f'http://localhost:{port}', timeout=2)
            if response.status_code == 200:
                jupyter_url = f"http://localhost:{port}"
                break
        except:
            pass
except:
    pass

if not jupyter_url:
    print("🚀 Starting Jupyter server...")
    subprocess.Popen([sys.executable, '-m', 'jupyter', 'notebook', '--no-browser', '--port=8888'],
                     stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    time.sleep(5)
    jupyter_url = "http://localhost:8888"

notebook_url = f"{jupyter_url}/notebooks/{notebook_name}"

print(f"🌐 Launching notebook: {notebook_url}")

try:
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=False, args=['--start-maximized'])
        context = browser.new_context(viewport={'width': 1920, 'height': 1080})
        page = context.new_page()
        page.goto(notebook_url, wait_until='networkidle', timeout=30000)
        print("✓ Notebook opened in browser for debugging!")
        print("💡 Close browser window when done")
        while browser.connected:
            time.sleep(1)
        if browser.connected:
            browser.close()
except Exception as e:
    print(f"Error: {e}")
    import webbrowser
    webbrowser.open(notebook_url)
